# DM (device manufacturer) connects to music (device) and assign them ownership - DM

## 1. Create Invitation from agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)


IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections


## 2. Create invitation

In [2]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection d6479df1-9ef0-493c-8dd9-bfbf6a33635a in State invitation
Connection ID d6479df1-9ef0-493c-8dd9-bfbf6a33635a
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'c51c7589-4363-4cb2-abc2-2e0cbeddbfba', 'label': 'Alice', 'recipientKeys': ['HSwsn6VAroBPox1xGmaLEchXW9Xybwb5VAWX74XfQTJb'], 'serviceEndpoint': 'http://192.168.65.3:8020'}
Connection Handler Called
Connection d6479df1-9ef0-493c-8dd9-bfbf6a33635a in State request


## 4. Accept Request - Recipient first!!!!!

In [3]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection d6479df1-9ef0-493c-8dd9-bfbf6a33635a in State response
ACCEPT REQUEST
{'state': 'response', 'rfc23_state': 'response-sent', 'created_at': '2023-01-05 11:53:05.793583Z', 'connection_id': 'd6479df1-9ef0-493c-8dd9-bfbf6a33635a', 'their_did': '7xVTb5Rj5S3kJBLmJ5YnvY', 'their_role': 'invitee', 'my_did': '5Jki8D2eQfhMw1F6cQorhi', 'invitation_key': 'HSwsn6VAroBPox1xGmaLEchXW9Xybwb5VAWX74XfQTJb', 'invitation_mode': 'once', 'updated_at': '2023-01-05 11:53:31.379460Z', 'routing_state': 'none', 'accept': 'manual', 'their_label': 'Bob'}
state response


In [4]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
connection_id = connection["connection_id"]

Trust Ping {'thread_id': 'de6575ae-3a8b-4281-b695-28f1154369ca'}
{'state': 'response', 'rfc23_state': 'response-sent', 'created_at': '2023-01-05 11:53:05.793583Z', 'connection_id': 'd6479df1-9ef0-493c-8dd9-bfbf6a33635a', 'their_did': '7xVTb5Rj5S3kJBLmJ5YnvY', 'their_role': 'invitee', 'my_did': '5Jki8D2eQfhMw1F6cQorhi', 'invitation_key': 'HSwsn6VAroBPox1xGmaLEchXW9Xybwb5VAWX74XfQTJb', 'invitation_mode': 'once', 'updated_at': '2023-01-05 11:53:31.379460Z', 'routing_state': 'none', 'accept': 'manual', 'their_label': 'Bob'}
Is Active? response
Connection Handler Called
Connection d6479df1-9ef0-493c-8dd9-bfbf6a33635a in State active


In [5]:
# response = await agent_controller.terminate()
# print(response)

None


# Give MUSIC the VC of ownership and origin - DM

## 1. Write Ownership schema

In [5]:
# Define you schema name - must be unique on the ledger
schema_name = "music_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["origin", "name", "time", "version", "author", "owner"]

# Write schema to ledger
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

# Write cred. def.
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)


# # Update schema
# attributes = ["author", "name", "time", "is_original", "type"]
# schema_version = "0.0.2"
# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# updated_schema_id = response["schema_id"]
# print(updated_schema_id)

# # Write another Credential Definition for the Updated Schema
# response = await agent_controller.definitions.write_cred_def(updated_schema_id, support_revocation=True)
# updated_cred_def_id = response["credential_definition_id"]
# print(updated_cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1
PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default


## 2. Write VC

In [6]:
nym = await agent_controller.wallet.get_public_did()
print(nym["result"]["did"])
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did='PQRXDxdGqQGSZ8z69p4xZP')
print(created_schema)
schema_id = created_schema["schema_ids"][-1]

PQRXDxdGqQGSZ8z69p4xZP
{'schema_ids': ['PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1']}


In [7]:
definitions = await agent_controller.definitions.search_created(schema_id=schema_id)
print(definitions["credential_definition_ids"])
cred_def_id = definitions["credential_definition_ids"][-1]

['PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default']


In [8]:
response = await agent_controller.connections.get_connections()
results = response['results']
print(len(results))
# print("Results : ", results)
if len(results) > 0:
    connection = response['results']
    for con in connection:
        print("Connection :", con)
        if con['state'] == 'active':       
            connection_id = con["connection_id"]
            print("Active Connection ID : ", connection_id)
        else: print("Inactive Connection ID: ", con["connection_id"], "with state: ", con["state"])
else:
    print("You must create a connection")

1
Connection : {'state': 'active', 'rfc23_state': 'completed', 'created_at': '2023-01-05 11:53:05.793583Z', 'connection_id': 'd6479df1-9ef0-493c-8dd9-bfbf6a33635a', 'their_did': '7xVTb5Rj5S3kJBLmJ5YnvY', 'their_role': 'invitee', 'my_did': '5Jki8D2eQfhMw1F6cQorhi', 'invitation_key': 'HSwsn6VAroBPox1xGmaLEchXW9Xybwb5VAWX74XfQTJb', 'invitation_mode': 'once', 'updated_at': '2023-01-05 11:53:32.370711Z', 'routing_state': 'none', 'accept': 'manual', 'their_label': 'Bob'}
Active Connection ID :  d6479df1-9ef0-493c-8dd9-bfbf6a33635a


In [9]:
import datetime
response = await agent_controller.wallet.create_did()
print(response)
music_did = response["result"]["did"]
verkey = response["result"]["verkey"]
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "author", "value": nym["result"]["did"]},
    {"name": "name", "value": music_did},
    {"name": "time", "value": time},
    {"name": "origin", "value": nym["result"]["did"]},
    {"name": "owner", "value": nym["result"]["did"]},
    {"name": "version", "value": "v1"}
]
print(credential_attributes)


# send credential
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

{'result': {'did': 'UmVQ6afzGd6oePeCshPvdT', 'verkey': 'G8mkcU6By5y6M7pBxKNu8TsKLDcG6iUioBBfuNiAKhTH', 'posture': 'wallet_only'}}
[{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'name', 'value': 'UmVQ6afzGd6oePeCshPvdT'}, {'name': 'time', 'value': '01-05-2023, 11:56:07'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'version', 'value': 'v1'}]
Credential exchange 4df882bb-7dd5-4aca-ad9f-ef5d2715f9e7, role: issuer, state: offer_sent


## 4. Wait for credential exchange

In [10]:
response = await agent_controller.issuer.get_records()
print(response['results'])
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

[{'state': 'credential_acked', 'created_at': '2023-01-05 11:56:07.853288Z', 'connection_id': 'd6479df1-9ef0-493c-8dd9-bfbf6a33635a', 'auto_remove': False, 'thread_id': 'f94fc381-bf4b-4e91-98e2-ea6b61a77f05', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default', 'key_correctness_proof': {'c': '10192130833627426586947702589679250098949171349435691191376900397976597707001', 'xz_cap': '261586222170142172902735024071118779259966076274987335194542550854122981163217419283349773387355200392096895339116318421548172199748097491198874626160827422691535391347693981990740968287330832643079574596343389032217551793262560913790316206353922565933192533172579864246583374700438899195204503560655766353006176312100513296239458594810232173985117409250047719230149142024622812467081383797853809900030314801894027055518190895143238903160169916375617135994135084218090837145020732624837067462096387485327380734163230050200029610597

In [11]:
## Remove records
response = await agent_controller.issuer.remove_record(record_id)
print(response)

{}


In [12]:
response = await agent_controller.terminate()
print(response)

None
